In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py

import os
# import wandb
import warnings

from config import *
from utils.eda_functions import *
from src.data_preprocessing.vitals_labs import *
# from src.data_preprocessing.split_dataset import *
from utils.safe_display import blind_display

%load_ext autoreload
%autoreload 2

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
import scipy.stats as ss

In [4]:
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

warnings.filterwarnings('ignore')

In [5]:
BLINDED = False
RANDOM = 0

In [6]:
patients_train = load_from_pickle(os.path.join(DATA_DIR, 'patients_train.pkl'))
patients_dev = load_from_pickle(os.path.join(DATA_DIR, 'patients_dev.pkl'))
patients_test = load_from_pickle(os.path.join(DATA_DIR, 'patients_test.pkl'))

vitals_train = load_from_pickle(os.path.join(DATA_DIR, 'vitals_train.pkl'))
vitals_dev = load_from_pickle(os.path.join(DATA_DIR, 'vitals_dev.pkl'))
vitals_test = load_from_pickle(os.path.join(DATA_DIR, 'vitals_test.pkl'))

interv_train = load_from_pickle(os.path.join(DATA_DIR, 'interv_train.pkl'))
interv_dev = load_from_pickle(os.path.join(DATA_DIR, 'interv_dev.pkl'))
interv_test = load_from_pickle(os.path.join(DATA_DIR, 'interv_test.pkl'))

Ys_train = load_from_pickle(os.path.join(DATA_DIR, 'Ys_train.pkl'))
Ys_dev = load_from_pickle(os.path.join(DATA_DIR, 'Ys_dev.pkl'))
Ys_test = load_from_pickle(os.path.join(DATA_DIR, 'Ys_test.pkl'))

In [7]:
static = pd.read_hdf(DATA_FILE_PATH,'patients')
static.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [8]:
idx = Ys_train.index.get_level_values('subject_id')

In [12]:
static.shape

(34472, 28)

In [11]:
len(idx)

16760

In [10]:
pat_idx = static.loc[idx]
pat_idx.shape

(16760, 28)